# Recommending Amazon Products using Graph Neural Networks in PyTorch Geometric

- Read README.MD to install the dependencies.

Based on https://wandb.ai/manan-goel/gnn-recommender/reports/Recommending-Amazon-Products-using-Graph-Neural-Networks-in-PyTorch-Geometric--VmlldzozMTA3MzYw

In [1]:
from tqdm import tqdm
import numpy as np
from IPython.display import IFrame
import os

import torch
from torch import nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric import utils
from torch_geometric.transforms import RandomLinkSplit
import torch_geometric as pyg

from pyvis.network import Network

In [10]:
from model import GNN

In [2]:
from torch_geometric.explain import Explainer, PGExplainer

In [28]:
from torch_geometric.loader.node_loader import NodeLoader
from torch_geometric.loader.neighbor_loader import NeighborLoader
from torch_geometric.sampler.base import NodeSamplerInput
from torch_geometric.sampler.neighbor_sampler import NeighborSampler

# Data

In [ ]:
# Save the splits and save as W&B artifacts
train_graph = torch.load('train.pt')
val_graph = torch.load('val.pt')
test_graph = torch.load('test.pt')

## Model

In [ ]:
# This notebook uses the pre-trained model
# -> generated by `Recommending Amazon Products-Model.ipynb`

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optim_wd = 0

#epochs = 300
epochs = 3

hidden_dim = 1024
dropout = 0.3
num_layers = 2
lr = 1e-5
node_emb_dim = 2  # 2 features: in/out-degrees
batch_size = 1024

In [ ]:
model_name = "gnn.pt"

model = GNN(
    node_emb_dim, 
    hidden_dim, 
    hidden_dim, 
    num_layers, 
    dropout
).to(device) # the graph neural network that takes all the node embeddings as inputs to message pass and agregate

# skip training if it is trained
model.load_state_dict(torch.load(model_name))
model.eval()

In [ ]:
train_graph = train_graph.to(device)
val_graph = val_graph.to(device)

## Explain model using PGExplainer

In [27]:
# from torch_geometric.explain.config import ModelConfig

In [29]:
pgexplainer = Explainer(
    model=model,
    algorithm=PGExplainer(epochs=epochs_explainer, lr=0.003),
    explanation_type='phenomenon',  # only accepts phenomenon explanation
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',  # PGExplainer only accepts node or graph-level
        return_type='log_probs',
    ),
)

In [30]:
# need to train with batches, otherwise raises memory errors

test_dataloader = NeighborLoader(
    train_graph,
    num_neighbors=[50] * 2,
    batch_size=1000,
    input_nodes=[1 for _ in range(train_graph.x.shape[0])],
    shuffle=True,
)

In [31]:
sampled_data = next(iter(test_dataloader))

In [32]:
sampled_data

Data(x=[1019, 2], edge_index=[2, 4021], pos_edge_label=[49041], pos_edge_label_index=[2, 49041], neg_edge_label=[49041], neg_edge_label_index=[2, 49041], n_id=[1019], e_id=[4021], input_id=[1000], batch_size=1000)

In [ ]:
# Train against a variety of node-level or graph-level predictions:
for epoch in tqdm(range(epochs_explainer)):
    for data in test_dataloader:
        N = data.x.shape[0]
        target = torch.tensor([1 for _ in range(N)])  # only positive examples (edge exists)
        indices = np.random.choice(list(range(N)), replace=False, size=int(.1 * N))  # train only on 10%
        # do I need to guarantee that `node_index` is in indeces??
        
        for index in indices:  # Indices to train against.
            # check again this training. I suppose there is something wrong here - specially the indices
            loss = pgexplainer.algorithm.train(
                epoch, 
                model, 
                torch.tensor(data.x).float().to(device), 
                data.edge_index,
                target=target, 
                index=[index],
            )

  0%|                                                                                                        | 0/10 [00:00<?, ?it/s]C:\Users\henri\AppData\Local\Temp\ipykernel_3948\1189303467.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data.x).float().to(device),


In [ ]:
# Get the final explanations:
explanation_pge = pgexplainer(
    torch.tensor(test_graph.x).float().to(device), 
    test_graph.edge_index, 
    index=0,
    target=target[:test_graph.x.shape[0]]
)

In [ ]:
print(explanation_pge.edge_mask)

In [ ]:
explanation_pge.get_explanation_subgraph()

In [ ]:
path_pge = 'pge-subgraph.pdf'
path_pge = None   # just show on cell
explanation_pge.visualize_graph()
print(f"Subgraph visualization plot has been saved to '{path_pge}'")

In [ ]:
# IFrame(path_pge, width=800, height=500)

In [ ]:
## Why the graph is blank?